In [2]:
import numpy as np
import pandas as pd
import os

from tqdm import tqdm_notebook as tqdm

from utils.ffmpeg_processor import ffmpegProcessor
from utils.audio import melspectrogram, show_melspectrogram, save_melspectrogram
from utils.encoder import FakeRealEncoder

Using TensorFlow backend.


In [3]:
dir_p0 = [ 'dfdc_train_part_1',
        'dfdc_train_part_2',
        'dfdc_train_part_3',
        'dfdc_train_part_4',
        'dfdc_train_part_5',
        'dfdc_train_part_6',
        'dfdc_train_part_7',
        'dfdc_train_part_8',
        'dfdc_train_part_9' ]
dir_p1 = [ 'dfdc_train_part_10',
        'dfdc_train_part_11',
        'dfdc_train_part_12',
        'dfdc_train_part_13',
        'dfdc_train_part_14',
        'dfdc_train_part_15',
        'dfdc_train_part_16',
        'dfdc_train_part_17',
        'dfdc_train_part_18',
        'dfdc_train_part_19' ]
dir_p2 = [ 'dfdc_train_part_20',
        'dfdc_train_part_21',
        'dfdc_train_part_22',
        'dfdc_train_part_23',
        'dfdc_train_part_24',
        'dfdc_train_part_25',
        'dfdc_train_part_26',
        'dfdc_train_part_27',
        'dfdc_train_part_28',
        'dfdc_train_part_29' ]
dir_p3 = [ 'dfdc_train_part_30',
        'dfdc_train_part_31',
        'dfdc_train_part_32',
        'dfdc_train_part_33',
        'dfdc_train_part_34',
        'dfdc_train_part_35',
        'dfdc_train_part_36',
        'dfdc_train_part_37',
        'dfdc_train_part_38',
        'dfdc_train_part_39' ]
dir_p4 = [ 'dfdc_train_part_40',
        'dfdc_train_part_41',
        'dfdc_train_part_42',
        'dfdc_train_part_43',
        'dfdc_train_part_44',
        'dfdc_train_part_45',
        'dfdc_train_part_46',
        'dfdc_train_part_47',
        'dfdc_train_part_48',
        'dfdc_train_part_49' ]


In [ ]:
wrk_dir = '../videos/'

dirs = ['dfdc_train_part_49']

vp = VideoProcessor()
le = FakeRealEncoder()

### Estrategia

* Video a video
* Extraemos caras de frames 
* Tenemos 300 frames
* 3 frames: 1 frame cada 10 frames
* 30 frames --> 10 muestras por video
* ¿y si está en movimiento?
* Nos quedamos con la capa H 
* comparamos con misma zona en video original
* Si es diferente, guardamos la muestra fake y la misma muestra real

In [ ]:
def extract_real_audios(dir, metadata, ap, le, n_chunks, len_chunks, inc_chunk):
    df = metadata.query('label == "REAL"')
    for index, row in tqdm(df.iterrows(), total=len(df)):
        ar = ap.extract_audio(f"../videos/{dir}/{row.video}")
        ar = np.where((ar > -noise_level)&(ar < noise_level), 0, ar)
        ar = np.resize(ar, (n_chunks,len_chunks))
        for i in range(n_chunks):
            save_melspectrogram(ar[i], 
                                f'REAL_{row.video}_{i}',
                                le.to_categorical(['REAL']))

In [ ]:
def extract_fake_audios(dir, metadata, ap, le, threshold, n_chunks, len_chunks, inc_chunk):
    df = metadata.query('label == "FAKE"')
    for index, row in tqdm(df.iterrows(), total=len(df)):
        af = ap.extract_audio(f"../videos/{dir}/{row.video}")
        af = np.where((af > -noise_level)&(af < noise_level), 0, af)
        af = np.resize(af, (n_chunks,len_chunks))
        
        ar = ap.extract_audio(f"../videos/{dir}/{row.original}")
        ar = np.where((ar > -noise_level)&(ar < noise_level), 0, ar)
        ar = np.resize(ar, (n_chunks,len_chunks))

        
        for i in range(n_chunks):
            if np.max(af[i]-ar[i]) > threshold:
                save_melspectrogram(af[i], 
                                    f'FAKE_{row.video}_{i}',
                                    le.to_categorical(['FAKE']))

In [ ]:
for d in dirs:
    metadata = pd.read_json('../videos/' + d + '/metadata.json').T
    metadata['video'] = metadata.index
    
    print(f'Extracting FAKE from {d}...')
    extract_fake_audios(d, metadata, ap, le, threshold, n_chunks, len_chunks, inc_chunk)
    
    print(f'Extracting REAL from {d}...')
    extract_real_audios(d, metadata, ap, le, n_chunks, len_chunks, inc_chunk)
    

Extracting FAKE from dfdc_train_part_49...



Extracting REAL from dfdc_train_part_49...


## Test

In [ ]:
from utils.utils import npz_in_folder
files = npz_in_folder('../data/audio')

In [ ]:
data = np.load(files[42])
log_S=data['data']
label=data['label']
data.close()

In [ ]:
log_S.shape, label.shape, label

In [ ]:
show_melspectrogram(log_S)